# Matching algorithm

Make sure that you followed the instructions in the "Preprocess and label images" notebook.

### Import

In [1]:
import os
import cv2
import numpy as np

from matplotlib import pyplot as plt

In [2]:
resized_images = "./data/resized/"
labels_folder = "BBox_Label_Tool/Labels/001/"
SHOW_IMAGES = False

### Link each image to its bounding boxes label

In [3]:
rects = {}
for file in os.listdir(labels_folder):
    with open(labels_folder + file, "r") as text_file: 
        lines = text_file.readlines()[1:]
        content = [l.replace("\n", "") for l in lines]
        labels = set([rect.split(" ")[-1] for rect in content])
        tmp = {}
        for lab in labels:
            tmp[lab] = [rect.split(" ")[:-1] for rect in content if lab in rect]
        rects[file[:-4]] =  tmp

data = {}
for img in os.listdir(resized_images):
    if img[-4:] == ".jpg":
        ID = img[:-4]
        image = cv2.imread(resized_images + img,1)

        data[ID] = {"image": image, "rects": rects[ID]}

### Crop the bounding boxes and save them in a folder

In [4]:
template_folder = {}
for ID in data.keys():
    image = data[ID]["image"].copy()
    for elem in data[ID]["rects"].keys():
        if not os.path.exists("./data/cropped_"+str(elem)+"/"):
            os.makedirs("./data/cropped_"+str(elem)+"/")
        
        i = 0
        for rect in data[ID]["rects"][elem]:
            x, y, w, h = [int(elem) for elem in rect]
            cropped = image[y:h, x:w]
            template_folder[elem] = "./data/cropped_"+str(elem)+"/"
            cv2.imwrite("./data/cropped_"+str(elem)+"/" + ID + str(i) + ".jpg",cropped)
            i+=1
            if SHOW_IMAGES:
                cv2.imshow('image',cropped)
                cv2.waitKey(0)
                cv2.destroyAllWindows()

### Fonction to merge overlapping boxes

In [5]:
def non_max_suppression_fast(boxes, overlapThresh):
    if len(boxes) == 0:
        return []

    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")

    pick = []
    
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]
    
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    
    idxs = np.argsort(y2)
    
    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
        
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])
    
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
    
        overlap = (w * h) / area[idxs[:last]]
    
        idxs = np.delete(idxs, np.concatenate(([last],
            np.where(overlap > overlapThresh)[0])))
    
    return boxes[pick].astype("int")

### Fonction to delete the "bad" boxes

In [6]:
def delete_non_overlapping(boxes, confidence):
    if len(boxes) == 0:
        return []
    
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")
    
    pick = []
    
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]

    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    
    idxs = np.argsort(y2)
    
    for i in idxs:
        idxs_without = np.delete(idxs, np.where(idxs==i)[0])
    
        xx1 = np.maximum(x1[i], x1[idxs_without])
        yy1 = np.maximum(y1[i], y1[idxs_without])
        xx2 = np.minimum(x2[i], x2[idxs_without])
        yy2 = np.minimum(y2[i], y2[idxs_without])
    
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
    
        overlap = (w * h) / area[idxs_without]
        if len(list(set(overlap))) > confidence:
            pick.append(i)
      
    return boxes[pick].astype("int")

In [ ]:

# All the 6 methods for comparison in a list
methods = ['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED',
                    'cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']
#meth = methods[1]


object_to_detect = "fiducial"
show = True
images = os.listdir(resized_images)


   
for image_file in images:

    if image_file[-4:] == ".jpg":
        rects = []
        #for meth in methods:
        meth = methods[0]
        img = cv2.imread(resized_images + image_file, 1)
        img_copy = img.copy()

        img2 = img.copy()
        img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

        for template_file in os.listdir(template_folder[object_to_detect]):
            if template_file[-4:] == ".jpg":
                template = cv2.imread(template_folder[object_to_detect] + template_file,0)
                w, h = template.shape[::-1]
                #img = img2.copy()
                method = eval(meth)

                # Apply template Matching
                res = cv2.matchTemplate(img2,template,method)
                min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

                # If the method is TM_SQDIFF or TM_SQDIFF_NORMED, take minimum
                if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
                    top_left = min_loc
                else:
                    top_left = max_loc
                bottom_right = (top_left[0] + w, top_left[1] + h)

                rects.append([top_left[0], top_left[1], bottom_right[0], bottom_right[1]])

                cv2.rectangle(img_copy,top_left, bottom_right, 255, 2)
        new_rect = []
        for r in rects:
            new_rect.append((r[0], r[1], r[2], r[3]))

        new_rect = np.array(new_rect)


        img_copy = img.copy()

        final_BB = delete_non_overlapping(new_rect, 1)
        final_BB = non_max_suppression_fast(final_BB,0.1)

        for r in final_BB:
            cv2.rectangle(img_copy,(r[0], r[1]), (r[2], r[3]), 255, 2)
        if SHOW_IMAGES:
            cv2.imshow('image',img_copy)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
    

        
